# Cái này là để cài đặt folder thành pakege để gọi imoport mà ko lỗi á. Nó tự sinh ra file 'signature_verification.egg-info' chứa thông tin tác giả của mìnhmình

In [1]:
%cd ..

# Cài đặt package
!pip install -e .

# Import module
from utils.helpers import load_config
from losses.contrastive_loss import contrastiveLoss
from models.triplet_network import TripletNetwork
from datasets import siamese_dataloader


print("Package installed and imported successfully!")

d:\Study\School\Phương pháp nghiên cứu\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///D:/Study/School/Ph%C6%B0%C6%A1ng%20ph%C3%A1p%20nghi%C3%AAn%20c%E1%BB%A9u/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for signature_verification (pyproject.toml): started
  Building editable for signature_verification (pypro


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: C:\Users\LEGION\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Package installed and imported successfully!


In [7]:
# pip install pyyaml

In [2]:
import os
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import cv2
import importlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
#Nayf ae xem sửa thửthử
config = load_config(r'D:\Study\School\Phương pháp nghiên cứu\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\configs\config_siamese.yaml')
print(config)

{'model': {'backbone': 'resnet34', 'feature_dim': 512}, 'training': {'batch_size': 32, 'learning_rate': 0.001, 'optimizer': 'adam', 'num_epochs': 50, 'margin': 0.5, 'scheduler': 'cosine'}, 'dataset': {'original_data_path': 'D:\\Study\\School\\Phương pháp nghiên cứu\\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\\data\\full_org', 'forgeries_data_path': 'D:\\Study\\School\\Phương pháp nghiên cứu\\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\\data\\full_forg', 'input_size': [220, 150]}, 'device': 'cuda', 'logging': {'log_dir': './logs/', 'checkpoint_dir': './checkpoints/', 'save_freq': 5}}


In [4]:
importlib.reload(siamese_dataloader)
train_loader, val_loader = siamese_dataloader.create_dataloaders(config)

NotADirectoryError: [WinError 267] The directory name is invalid: 'D:\\Study\\School\\Phương pháp nghiên cứu\\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\\data\\full_org\\Thumbs.db'

In [ ]:
model = TripletNetwork(
    backbone=config['model']['backbone'],
    feature_dim=config['model']['feature_dim']
)
optimizer = optim.Adam(model.parameters(), lr=config['training']['learning_rate'])
device = torch.device(config['device'] if torch.cuda.is_available() else "cpu")
model.to(device)

AE xem thử rồi cấu trúc ổn không.

In [ ]:
#Train kiểu kiểu như vầy ae cứ xem tham thảo

num_epochs = config['train']['num_epochs']
checkpoint_dir = config['logging']['checkpoint_dir']
save_freq = config['logging']['save_freq']

# Tạo thư mục lưu checkpoint nếu chưa có
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for (img1, img2), labels in train_loader:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

        output1 = model(img1)
        output2 = model(img2)

        loss = contrastiveLoss(output1, output2, labels, margin=config['loss']['margin'])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}] - Loss: {avg_loss:.4f}")

    if epoch % save_freq == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")